In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import nibabel as nib
from os import listdir
from os.path import isfile, join
from scipy import signal
import sys

# current path
cur_path = os.getcwd()
# file name and path
file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
file_name = cur_path + "/" + str(file) + "/dfMRI_raw.nii.gz"
# loading nifti object/header
raw_data_ = nib.load(file_name)
header  = raw_data_.header
raw_data = raw_data_.get_fdata()

### 1. Motion correction
#### 1.1 Split b0 and b1 values

In [29]:
def gzip(filename, zip):
    """
    Function that zips or unzips the filename file
    Input:
        filename : str
        zip : bool, true -> zip, false -> unzip
    Ouput:
        new_filename : str
    """
    if zip == False:
        new_filename = filename.replace('.gz', '')
        os.system(f"gzip -f -c -d {filename} > {new_filename}")
    elif zip == True:
        os.system(f"gzip {filename} -f")
        new_filename = f'{filename}.gz'
    else:
        new_filename = None
        
    return new_filename


In [30]:
import nipype.interfaces.spm as spm
import zipfile

dfmri_degibbs_sdc = nib.load(f'{cur_path}/data/topup/dfmri_degibbs_sdc.nii.gz')
dfmri_degibbs_sdc = dfmri_degibbs_sdc.get_fdata()
for i in range(2):
    dfmri_degibbs_sdc_b = dfmri_degibbs_sdc[...,np.arange(i, dfmri_degibbs_sdc.shape[3], 2)]
    img_deggibs_sdc_b = nib.Nifti1Image(dfmri_degibbs_sdc_b, np.eye(4))
    nib.save(img_deggibs_sdc_b, f'{cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}.nii.gz') 

    gzip(f'{cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}.nii.gz', zip=False)
 

### 1.2 Apply spm for motion correction 
#### Problem : I don't have a matlab licence

In [27]:
# Matlab is needed
realign = spm.Realign()
print(type(f"{cur_path}/data/ADC/dfmri_degibbs_sdc_b0.nii.gz"))
realign.inputs.in_files = f"{cur_path}/data/ADC/dfmri_degibbs_sdc_b0.nii"
realign.run()

<class 'str'>


TraitError: Each element of the 'in_files' trait of a RealignInputSpec instance must be a pathlike object or string representing an existing file or a list of items which are a pathlike object or string representing an existing file, but a value of 'dfmri_degibbs_sdc_b0.nii.gz' <class 'str'> was specified.

### 1.3 Take the output from Wictor "rdfMRI_denoised_unringed_sdc_b_{i}.nii.gz" to do the ANTS rigid registration

In [3]:
# Since I don't have matlab, I take Wictor's output here
for i in range(2):
    rdfMRI_sdc_b = nib.load(f'/home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i}.nii.gz')
    rdfMRI_sdc_b = rdfMRI_sdc_b.get_fdata()

    rdfMRI_sdc_b_mean = np.mean(rdfMRI_sdc_b, axis=3)
    img_rdfMRI_sdc_b_mean = nib.Nifti1Image(rdfMRI_sdc_b_mean, np.eye(4)) 
    nib.save(img_rdfMRI_sdc_b_mean, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i}_mean.nii.gz') 

### 1.4 Ants rigid registration, fixed image = rdfMRI_denoised_unringed_sdc_b_0_mean

In [9]:
os.system('antsRegistration --dimensionality 3 --float 0 '
        f' --output [{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_0, {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_0.nii] '
        ' --interpolation Linear '
        ' --winsorize-image-intensities [0.01,0.99] ' 
        ' --use-histogram-matching 0 '
        ' --transform Rigid[0.0001] ' 
        f' --metric MI[{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_0_mean.nii.gz, {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean.nii.gz,1,32,Regular,0.25] '
        ' --convergence [10000x10000x10000x10000,1e-6,10] '
        ' --shrink-factors 4x4x2x1 ' 
        ' --smoothing-sigmas 2x2x1x0vox')

0

### 1.5 Apply the transform to rdfMRI_denoised_unringed_sdc_b_1

In [17]:
os.system(f'antsApplyTransforms -r {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_0_mean.nii.gz'
          f' -i {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1.nii.gz -e 3 '
          f'-t {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_00GenericAffine.mat '
          f'-v -o {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1.nii.gz')

Using double precision for computations.
Input time-series image: /home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_1.nii.gz
Reference image: /home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_0_mean.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_00GenericAffine.mat (type = AffineTransform)
Default pixel value: 0
Interpolation type: LinearInterpolateImageFunction
  Applying transform(s) to timePoint/channel/dimension5 0 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 1 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 2 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 3 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 4 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 5 (out of 300).
  Applying tr

0

### 1.6 Put together b0 and b1 again

In [20]:
rdfMIR_denoised_unringed_sdc = np.zeros(header['dim'][1:5])

for i in range(2):
    rdfMIR_denoised_unringed_sdc_b = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i}.nii.gz')
    rdfMIR_denoised_unringed_sdc_b = rdfMIR_denoised_unringed_sdc_b.get_fdata()
    rdfMIR_denoised_unringed_sdc[..., np.arange(i, rdfMIR_denoised_unringed_sdc.shape[3], 2)] = rdfMIR_denoised_unringed_sdc_b

del rdfMIR_denoised_unringed_sdc_b
img_rdfMIR_denoised_unringed_sdc = nib.Nifti1Image(rdfMIR_denoised_unringed_sdc, np.eye(4), header) 
nib.save(img_rdfMIR_denoised_unringed_sdc, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc.nii.gz') 

In [30]:
# Replace NaN values by 0
rdfMIR_denoised_unringed_sdc = np.nan_to_num(rdfMIR_denoised_unringed_sdc)

assert np.argwhere(np.isnan(rdfMIR_denoised_unringed_sdc)).size == 0, "There are still NaN values in the array"

img_rdfMIR_denoised_unringed_sdc = nib.Nifti1Image(rdfMIR_denoised_unringed_sdc, np.eye(4), header) 
nib.save(img_rdfMIR_denoised_unringed_sdc, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc.nii.gz') 
